# Imports

In [47]:
# Freqtrade Workspace

# Notebook Helper
%load_ext autoreload
%autoreload 2
%env WANDB_SILENT=true

# Standard Imports
from pathlib import Path
from typing import List, Callable, Tuple, Any
from wandb.wandb_run import Run
from datetime import datetime, timedelta
from functools import reduce
import attr
import gc
import os
import wandb
import nest_asyncio
import logging
import sys
import pandas as pd
import numpy as np
import mplfinance as mpf
import pandas_ta as ta
import talib.abstract as ta
import random

# Resolve CWD
gc.collect()
nest_asyncio.apply()

while "freqtrade" not in os.listdir():
    os.chdir("..")
    if "freqtrade" in os.listdir():
        nest_asyncio.apply()
        logger = logging.getLogger("freqtrade.nbtools")
        handler = logging.StreamHandler(stream=sys.stdout)
        handler.setFormatter(logging.Formatter("%(name)s - %(message)s"))
        logger.addHandler(handler)
        logger.setLevel(logging.DEBUG)

# Freqtrade Imports
from freqtrade.optimize.optimize_reports import text_table_add_metrics
from freqtrade.ml.lightning import LightningModule, LightningConfig
from freqtrade.ml.trainer import TradingTrainer
from freqtrade.ml.container import LightningContainer
from freqtrade.ml import loader, lightning_utils
from freqtrade.nbtools.preset import LocalPreset, ConfigPreset, FilePreset, CloudPreset
from freqtrade.nbtools.backtest import backtest
from freqtrade.nbtools import plotting, configs
from freqtrade.nbtools.helper import free_mem
from freqtrade.nbtools.pairs import PAIRS_HIGHCAP_NONSTABLE

import freqtrade.vendor.qtpylib.indicators as qtpylib

# Constants
PATH_MOUNT = Path.cwd().parent / "mount"
PATH_DATA = PATH_MOUNT / "data"
PATH_PRESETS = PATH_MOUNT / "presets"
PATH_STRATEGIES = PATH_PRESETS / ".strategy_files"

# Define custom functions
def get_random_index(dataframe: pd.DataFrame, loc_condition):
    if len_buy == 0: raise Exception("Zero BUY signals")
    if len_sell == 0: raise Exception("Zero SELL signals")
    idx = 0
    while idx < 1000 or idx > len(dataframe) - 1000:
        idx = dataframe.loc[loc_condition].sample()["idx"][0]
    return idx


def get_middle(dataframe: pd.DataFrame, index: int, num_look: int) -> pd.DataFrame:
    return dataframe.loc[
        (dataframe["idx"] >= index - num_look) & (dataframe["idx"] <= index + num_look)
    ]


style = {'base_mpl_style': 'fast',
         'marketcolors'  : {'candle': {'up': '#00b060', 'down': '#fe3032'},
                            'edge'  : {'up': '#00b060', 'down': '#fe3032'},
                            'wick'  : {'up': '#606060', 'down': '#606060'},
                            'ohlc'  : {'up': '#00b060', 'down': '#fe3032'},
                            'volume': {'up': '#4dc790', 'down': '#fd6b6c'},
                            'vcedge': {'up': '#1f77b4', 'down': '#1f77b4'},
         'vcdopcod'      : True,
         'alpha'         : 0.9},
         'mavcolors'     : None,
         'facecolor'     : '#fafafa',
         'gridcolor'     : '#d0d0d0',
         'gridstyle'     : '-',
         'y_on_right'    : True,
         'rc'            : {'axes.labelcolor': '#101010',
                            'axes.grid.axis' : 'y',
                            'ytick.color'    : '#101010',
                            'xtick.color'    : '#101010',
                            'figure.titlesize': 'x-large',
                            'figure.titleweight':'semibold',
                            'axes.edgecolor'  : 'black',
                           'axes.linewidth'  :  1.0,
                           'lines.linewidth' :  1.0,
                           },
         'base_mpf_style': 'yahoo'}


def plot(dataframe: pd.DataFrame, display_indicators: dict, draw_signals: bool, length = 150):
    """ Plot the price chart!
    """
    df = dataframe.copy()[-length:]
    
    addplot = []
    kwargs = {}
    
    if draw_signals:
        df.loc[df["buy"] == 1, "buy"] = df["low"] - (0.0025*df["low"])
        df.loc[df["buy"] == 0, "buy"] = np.nan
        df.loc[df["sell"] == 1, "sell"] = df["high"] + (0.0025*df["high"])
        df.loc[df["sell"] == 0, "sell"] = np.nan
        
        # Add buy and sell markers
        if len(df.loc[df["buy"] > 0]) > 0: 
            addplot.append(mpf.make_addplot(df["buy"], type='scatter', markersize=100, marker='^', color="green"))
        if len(df.loc[df["sell"] > 0]) > 0: 
            addplot.append(mpf.make_addplot(df["sell"], type='scatter', markersize=100, marker='v', color="red"))
    
        # Add vline markers
        df.index = pd.to_datetime(df.index)
        kwargs["vlines"] = {
            "vlines": list(df.loc[df["buy"] > 0].index) + list(df.loc[df["sell"] > 0].index),
            "alpha": 0.25,
            "colors": ["green" for _ in range(len(df.loc[df["buy"] > 0]))] + ["red" for _ in range(len(df.loc[df["sell"] > 0]))]
        }
    
    for indicator, opt in display_indicators.items():
        _kwargs = {}
        
        if opt["type"] != "line":
            _kwargs["type"] = opt["type"]
        
        plot = mpf.make_addplot(
            df[indicator], 
            color=opt["color"], 
            panel=opt["panel"], 
            ylabel=indicator, 
            **_kwargs
        )
        
        addplot.append(plot)
    
    mpf.plot(
        df, 
        type="candle", 
        style=style, 
        figsize=(15, 8), 
        addplot=addplot,
        **kwargs,
    )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: WANDB_SILENT=true


# Build Any Strategy

In [48]:
df_load = loader.load_df(PATH_DATA / "binance"/ "BTC_USDT-5m.json", "5m").iloc[-50000:]
df_load = df_load.set_index("date")
df_load["idx"] = range(0, len(df_load))
df_load["buy"] = 0
df_load["sell"] = 0
df_load

,open,high,low,close,volume,idx,buy,sell
date,,,,,,,,
2021-02-24 23:25:00+00:00,49315.77,49444.41,49180.01,49309.66,193.292256,0,0,0
2021-02-24 23:30:00+00:00,49309.66,49514.61,49309.66,49355.46,237.686205,1,0,0
2021-02-24 23:35:00+00:00,49358.34,49739.32,49333.53,49552.84,330.522887,2,0,0
2021-02-24 23:40:00+00:00,49552.85,49599.74,49434.36,49555.10,170.640169,3,0,0
2021-02-24 23:45:00+00:00,49555.10,49747.49,49515.46,49653.58,164.437119,4,0,0
...,...,...,...,...,...,...,...,...
2021-08-17 13:40:00+00:00,46496.12,46564.42,46408.00,46564.41,113.297325,49995,0,0
2021-08-17 13:45:00+00:00,46564.42,46581.00,46481.61,46530.84,127.722063,49996,0,0
2021-08-17 13:50:00+00:00,46530.84,46530.84,46419.47,46445.62,186.850551,49997,0,0


Buy signals: 49842 | Sell signals: 49842
Index BUY: 10788 | Index SELL: 47231
Showing SELL example


In [52]:
def populate_indicators(dataframe: pd.DataFrame) -> pd.DataFrame:
    """ Populate Freqtrade Indicators """
    dataframe["macd"], dataframe["macdsignal"], dataframe["macdhist"] = ta.MACD(
        dataframe["close"], fastperiod=12, slowperiod=26, signalperiod=9
    )
    dataframe["rsi"] = ta.RSI(dataframe["close"], timeperiod=14)
    return dataframe


def populate_buy_trend(dataframe: pd.DataFrame) -> pd.DataFrame:
    """ Generate BUY Signals"""
    conditions = [
        dataframe['volume'] > 0,
    ]
            
    dataframe.loc[reduce(lambda x, y: x & y, conditions), 'buy'] = 1
    return dataframe

    
def populate_sell_trend(dataframe: pd.DataFrame) -> pd.DataFrame:
    """ Generate SELL Signals"""
    conditions = [
        dataframe['volume'] > 0,
    ]
            
    dataframe.loc[reduce(lambda x, y: x & y, conditions), 'sell'] = 1
    return dataframe

# Ignore
df_pop = populate_sell_trend(populate_buy_trend(populate_indicators(df_load.copy())))
look_len = 70
len_buy, len_sell = len(df_pop.loc[df_pop["buy"] == 1]), len(df_pop.loc[df_pop["sell"] == 1])
print(f"Buy signals: {len_buy} | Sell signals: {len_sell}")
if len_buy == 0: raise Exception("Zero BUY signals")
if len_sell == 0: raise Exception("Zero SELL signals")
idx_buy, idx_sell = get_random_index(df_pop, df_pop["buy"] == 1), get_random_index(df_pop, df_pop["sell"] == 1)
print(f"Index BUY: {idx_buy} | Index SELL: {idx_sell}")
if random.randint(0, 1) == 0: 
    print("Showing SELL example")
    df_middle = get_middle(df_pop, idx_sell, look_len)
else:
    print("Showing BUY example")
    df_middle = get_middle(df_pop, idx_buy, look_len)

# Start modify again
display_indicators = {
#     "macd":          {"panel": 1, "color": "blue", "type": "line"},
#     "macdsignal":    {"panel": 1, "color": "red", "type": "line"},
#     "macdhist":      {"panel": 1, "color": "green", "type": "bar"},
    "rsi":           {"panel": 1, "color": "purple", "type": "line"},
}

# Override df_middle if you want to see indicators only
df_middle = get_middle(df_pop, 40000, look_len)
plot(df_middle, display_indicators, draw_signals=False)

In [58]:
df_viz = df_load.copy()
indicator_col = "visualize"

# Populate your indicator here
df_viz[indicator_col] = ta.TEMA(df_viz["close"], timeperiod=14)

# Config visualize
ind = {indicator_col: {"panel": 0, "color": "purple", "type": "line"}}

df_viz = get_middle(df_viz, 40000, look_len)
plot(df_viz, ind, draw_signals=False)